In [1]:
import torch
from matplotlib import pyplot as plt
import cv2
from PIL import Image
import numpy as np
import pandas as pd
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16

# Model
model = torch.hub.load('/Users/hongjowell/parking2/yolov5', 'custom', path='runs/train/exp16/weights/best.pt', source='local')
model.classes = [0]

cam = cv2.VideoCapture(0)

cv2.namedWindow("parking lot")

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("parking lot", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = "/Users/hongjowell/parking/parkinglot.png"
        cv2.imshow(img_name, frame)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))

cam.release()
cv2.destroyAllWindows()


# Images
#dir = 'https://github.com/ultralytics/yolov5/raw/master/data/images/'
width = 1280
height = 720
dim = (width, height)
img = cv2.imread(img_name, cv2.IMREAD_UNCHANGED)
resized = cv2.resize(img, dim)


# Inference
results = model(resized)

# Results
results.print()  
results.save(labels=False)  # or .show()
#results.show()

# Data
#print(results.xyxy[0])  # print img1 predictions (pixels)
#                   x1           y1           x2           y2   confidence        class
# tensor([[7.50637e+02, 4.37279e+01, 1.15887e+03, 7.08682e+02, 8.18137e-01, 0.00000e+00],
#         [9.33597e+01, 2.07387e+02, 1.04737e+03, 7.10224e+02, 5.78011e-01, 0.00000e+00],
#         [4.24503e+02, 4.29092e+02, 5.16300e+02, 7.16425e+02, 5.68713e-01, 2.70000e+01]])

/opt/anaconda3/envs/parking/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5 🚀 v6.1-111-gb7faeda torch 1.11.0 CPU

Fusing layers... 
Model summary: 213 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


/Users/hongjowell/parking/parkinglot.png written!
Escape hit, closing...


image 1/1: 720x1280 (no detections)
Speed: 9.3ms pre-process, 287.2ms inference, 17.6ms NMS per image at shape (1, 3, 384, 640)
Saved 1 image to runs/detect/exp48


In [2]:
temp=results.pandas().xyxy[0]
temp1= pd.DataFrame(temp)

In [3]:
numpy_array = temp1.to_numpy()
np.savetxt("coordinates.txt", numpy_array, fmt = "%s", delimiter=' ')

In [4]:
filename = 'coordinates.txt'
data = np.genfromtxt(filename, delimiter=' ', dtype=None, names=('x1', 'y1', 'x2', 'y2', 'conf', 'class','name'))
print(data)

[]


/opt/anaconda3/envs/parking/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: genfromtxt: Empty input file: "coordinates.txt"
  


In [5]:
model1 = tf.keras.models.load_model("/Users/hongjowell/parking/Model/saved_model.h5")
HEIGHT = 49
WIDTH = 37
occupied = int(0)
empty = int(0)

2022-05-30 11:49:04.377001: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
from time import time
import cv2

# Create a new VideoCapture object
camera = cv2.VideoCapture(0)

# Initialise variables to store current time difference as well as previous time call value
previous = time()
delta = 0
while True:
    # Get the current time, increase delta and update the previous variable
    current = time()
    delta += current - previous
    previous = current
    
    if delta > 60:
        take_pic = camera.read()
        cv2.imwrite('/Users/hongjowell/parking/test.png', take_pic)
        img_name1 = cv2.imread('/Users/hongjowell/parking/test.png' , cv2.IMREAD_UNCHANGED)
        img1 = cv2.resize(img_name1, dim)

        info1 = []

        with open(filename) as input_file:
    
            for line in input_file:
                x1, y1, x2, y2, conf, labels, name = (
                    item.strip() for item in line.rsplit(' ',6))
                info1.append (dict(zip(('labels', 'conf', 'x1', 'y1', 'x2', 'y2'),
                                (labels, conf, x1, y1, x2, y2))))
                
        for one_info in info1:
            x1 = int(float(one_info["x1"]))
            y1 = int(float(one_info["y1"]))
            x2 = int(float(one_info["x2"]))
            y2 = int(float(one_info["y2"]))
            cropped_image = img1 [y1:y2, x1:x2]
            image = cv2.resize(cropped_image, (WIDTH, HEIGHT))
            image_x = np.expand_dims(image, axis=0)
            image_x = tf.keras.applications.vgg16.preprocess_input(image_x)
            prediction = model1.predict(image_x)
            prediction = np.squeeze(prediction)

            if prediction> 0.8:
                cv2.rectangle(img1, (x1, y1), (x2, y2),(255, 0, 0), 3)
                occupied = occupied + 1
        
            else:
                cv2.rectangle(img1, (x1, y1), (x2, y2),(0, 255, 0), 3)
                empty = empty + 1
      
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img1, str(occupied),
                (0, 30), font,
                1, (0, 255, 255),
                4, cv2.LINE_AA)
        cv2.putText(img1, str(empty),
                (0, 60), font,
                1, (0, 255, 255),
                4, cv2.LINE_AA)
        cv2.imshow("view", img1)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        delta = 0